In [ ]:
import requests
import csv
from datetime import datetime
import time

# Create a session to reuse HTTP connections
session = requests.Session()

def get_zones(api_token, page, per_page):
    url = "https://api.cloudflare.com/client/v4/zones"
    headers = {
        "Authorization": f"Bearer {api_token}",
    }
    params = {
        "page": page,
        "per_page": per_page
    }
    retries = 3
    for _ in range(retries):
        response = session.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if data['success'] and data['result']:
                # Filter zones for only DXP customers
                ZN_zones = [zone for zone in data['result'] if zone['account']['name'] == 'Input your account you wish to download']
                return True, ZN_zones
        else:
            print("Failed to fetch zones:", response.text)
            time.sleep(5)  # Wait for 5 seconds before retrying
    return False, None

def write_hostnames_to_csv(ZN_zones):
    current_date = datetime.now().strftime("%Y-%m-%d")
    filename = f'customer_hostnames_{current_date}.csv'
    with open(filename, mode='a', newline='') as file:  # Change 'w' to 'a' to append data to the file
        writer = csv.writer(file)
        for zone in ZN_zones:
            writer.writerow([zone['name']])
            print(f"Hostname: {zone['name']}")

# Example usage
api_token = 'Your API KEY'

page = 1
per_page = 1000
retry_delay = 1  # Initial delay time
all_zones = []

while True:
    success, zones = get_zones(api_token, page, per_page)
    if success:
        if zones:
            all_zones.extend(zones)
            write_hostnames_to_csv(zones)  # Write to CSV immediately after fetching
            page += 1
            retry_delay = 1  # Reset the retry delay on successful fetch
            # Introduce a delay between requests to avoid rate limit issues
            time.sleep(1)
        else:
            print("No more zones to process.")
            break
    else:
        # Exponential backoff: Increase delay time exponentially upon rate limit errors
        retry_delay *= 2  # Double the delay time
        print(f"Retrying in {retry_delay} seconds due to errors.")
        time.sleep(retry_delay)
        if retry_delay > 60:  # Maximum delay threshold to prevent infinite looping
            print("Maximum retry delay reached. Exiting.")
            break

# Optionally write all accumulated DXP customer hostnames to CSV at the end
# This will avoid writing duplicate entries if the above already writes to the CSV
# write_hostnames_to_csv(all_zones)
